# Amazon SageMaker で DeOldify を動かしてモノクロ画像をカラーにする 

このノートブックでは、モノクロ画像をカラー化するモデル [DeOldify](https://github.com/jantic/DeOldify) の学習済みモデルを Amazon SageMaker Processing を使って実行します。あらかじめ Amazon S3 にモノクロ画像を保存しておき、Processing Job 実行時にそのパスを指定することで、指定されたパスの中のモノクロ画像が全てカラー化されて Amazon S3 に保存されます。

## 準備
**このサンプルでは、カスタムコンテナを Amazon ECR に push する必要があります。**以下の操作でこのノートブックインスタンスで使用している IAM ロールに Amazon ECR にイメージを push するための権限を追加してください。

1. Amazon SageMaker コンソールからこのノートブックインスタンスの詳細画面を表示<br>
（左側のメニューのインスタンス -> ノートブックインスタンス -> インスタンス名をクリック）
1. 「アクセス許可と暗号化」の「IAM ロール ARN」のリンクをクリック（IAM のコンソールに遷移します）
1. 「ポリシーをアタッチします」と書いてある青いボタンをクリック
1. 検索ボックスに ec2containerregistry と入力し AmazonEC2ContainerRegistryFullAccess のチェックボックスをチェックする
1. 「ポリシーのアタッチ」と書いてある青いボタンをクリック


以下のセルでは、Amazon SageMaker を使うためのセットアップを行います。ロールの情報、ノートブックインスタンスのリージョン、アカウントID などの情報を取得しています。

In [ ]:
%matplotlib inline

import boto3
import sys
import sagemaker
import numpy as np
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
session = sagemaker.Session()
s3_output = session.default_bucket()
s3_prefix = 'deoldify-BYO'

## Amazon SageMaker Processing で DeOldify を実行

まずは Processing Job で使用する Docker コンテナイメージを作成します。必要なファイルは wget や git clone で取得してコンテナイメージの中に入れておきます。

In [ ]:
!mkdir -p docker-proc

In [ ]:
%%writefile docker-proc/Dockerfile

FROM nvcr.io/nvidia/pytorch:19.04-py3
    
ENV DEBIAN_FRONTEND=noninteractive
ENV TZ=Asia/Tokyo 

RUN apt-get -y update && apt-get install -y \
	python3-pip \
	software-properties-common \
	wget \
	ffmpeg \
	git

RUN mkdir -p /root/.torch/models

RUN mkdir -p /data/models

RUN mkdir -p /data/gitrepo
RUN cd /data/gitrepo && git clone https://github.com/jantic/DeOldify.git

RUN wget -O /root/.torch/models/vgg16_bn-6c64b313.pth https://download.pytorch.org/models/vgg16_bn-6c64b313.pth

RUN wget -O /root/.torch/models/resnet34-333f7ec4.pth https://download.pytorch.org/models/resnet34-333f7ec4.pth

RUN pip install --upgrade pip \
	&& pip install versioneer==0.18 \
		tensorboardX==1.6 \
		Flask==1.1.1 \
		pillow==6.1 \
		numpy==1.16 \
		scikit-image==0.15.0 \
		requests==2.21.0 \
		ffmpeg-python==0.2.0 \
		youtube-dl>=2019.4.17 \
		jupyterlab==1.2.4 \
		opencv-python>=3.3.0.10 \
		fastai==1.0.51

ADD . /data/

WORKDIR /data

# force download of file if not provided by local cache
RUN [[ ! -f /data/models/ColorizeArtistic_gen.pth ]] && wget -O /data/models/ColorizeArtistic_gen.pth https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
RUN [[ ! -f /data/models/ColorizeVideo_gen.pth ]] && wget -O /data/models/ColorizeVideo_gen.pth https://data.deepai.org/deoldify/ColorizeVideo_gen.pth

EXPOSE 8888
EXPOSE 5000

ENV PYTHONUNBUFFERED=TRUE
ENTRYPOINT ["python3"]


In [ ]:
ecr_repository = 'deoldify-byo-proc'
tag = ':latest'
uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

コンテナイメージを build して Amazon ECR に push します。

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker-proc
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

上記セルでコンテナイメージを build する際に no space left というエラーが出たら以下のセルのコメントを外して実行してください。

In [ ]:
# !docker system prune -a -f

以下のセルは DeOldify を使って、指定された Amazon S3 パスに保存されているモノクロ画像をカラー化して保存するスクリプトです。カラー化した画像は指定された Amazon S3 パスにアップロードされます。

In [ ]:
%%writefile preprocessing.py


import sys
sys.path.append('/data/gitrepo/DeOldify')

import glob


import numpy as np
import os
import pandas as pd
import argparse
import shutil
    
from deoldify import device
from deoldify.device_id import DeviceId
import torch
from os import path
import fastai
from deoldify.visualize import *
import warnings
from pathlib import Path
torch.backends.cudnn.benchmark=True

print(sys.version)

if __name__=='__main__':
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--input-dir', type=str, default=None)
    parser.add_argument('--output-dir', type=str, default=None)
    parser.add_argument('--render-factor', type=str, default='35')
    args, _ = parser.parse_known_args()
    
    #choices:  CPU, GPU0...GPU7
    device.set(device=DeviceId.GPU0)

    if not torch.cuda.is_available():
        print('GPU not available.')

    warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")
    
    colorizer = get_image_colorizer(artistic=True)
    colorizer_video = get_video_colorizer()
    
    file_list = glob.glob(args.input_dir + '/**')
    
    render_factor = int(args.render_factor)  #@param {type: "slider", min: 7, max: 40}
    watermarked = False #@param {type:"boolean"}
    
    for f in file_list:
        
        print('file: ' + os.path.basename(f) + ' is processing.')
        root, ext = os.path.splitext(f)

        if f is not None and f !='':
            if ext in ['.jpg', '.jpeg', '.png']:
                colorizer.plot_transformed_image(f, results_dir=Path(args.output_dir), render_factor=render_factor, display_render_factor=True, figsize=(8,8))
            else:
                print(f + ' is not image file.')
        else:
            print('Provide an image url and try again.')

    print('====results====')
    print(glob.glob(args.output_dir + '/**'))

作成したコンテナイメージとスクリプトを使って DeOldify を実行します。`ScriptProcessor` を作成する際に、使用するインスタンスタイプを指定します。

In [ ]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=processing_repository_uri,
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.c5.4xlarge')
#                                    instance_type='local')

すべてのセットアップが終わったら Processing Job を実行します。

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from time import gmtime, strftime 

processing_job_name = "deoldify-byo-process-{}".format(strftime("%d-%H-%M-%S", gmtime()))
output_destination = 's3://{}/{}/data'.format(s3_output, s3_prefix)
input_s3 = 's3://data-for-experiments/images/monochrome-images/'

local_input_path = '/opt/ml/processing/input/data'
local_output_path = '/opt/ml/processing/output'

script_processor.run(code='preprocessing.py',
                      job_name=processing_job_name,
                      inputs=[ProcessingInput(
                        source=input_s3,
                        destination=local_input_path)],
                      outputs=[ProcessingOutput(output_name='output',
                                                destination='{}/{}'.format(output_destination, processing_job_name),
                                                source=local_output_path)],
                      arguments=[
                          '--input-dir',local_input_path,
                          '--output-dir',local_output_path,
                          '--render-factor',"35"
                      ]
                    )

preprocessing_job_description = script_processor.jobs[-1].describe()